In [5]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [6]:
names=open('yob2004.txt').read().splitlines()
for i in range(len(names)):
    s=""
    for j in names[i].lower():
        if j==",":
            names[i]=s
            break
        else:
            s+=j
s=""
for i in names:
    for j in i:
        s+=j
s=['.']+sorted(set(s))


# Creating A Dictionary For Easy Access of Indices Of Alphabets
itos={ i:j for (i,j)in enumerate(s)}
stoi={itos[i]:i for i in itos.keys()}
print(names)

['emily', 'emma', 'madison', 'olivia', 'hannah', 'abigail', 'isabella', 'ashley', 'samantha', 'elizabeth', 'alexis', 'sarah', 'grace', 'alyssa', 'sophia', 'lauren', 'brianna', 'kayla', 'natalie', 'anna', 'jessica', 'taylor', 'chloe', 'hailey', 'ava', 'jasmine', 'sydney', 'victoria', 'ella', 'mia', 'morgan', 'julia', 'kaitlyn', 'rachel', 'katherine', 'megan', 'alexandra', 'destiny', 'jennifer', 'allison', 'savannah', 'maria', 'nicole', 'haley', 'mackenzie', 'brooke', 'makayla', 'trinity', 'kylie', 'kaylee', 'paige', 'faith', 'lily', 'zoe', 'stephanie', 'jenna', 'andrea', 'riley', 'katelyn', 'angelina', 'kimberly', 'mary', 'madeline', 'leah', 'lillian', 'michelle', 'amanda', 'sara', 'sofia', 'rebecca', 'jordan', 'alexa', 'gabrielle', 'caroline', 'vanessa', 'gabriella', 'avery', 'marissa', 'ariana', 'audrey', 'jada', 'autumn', 'maya', 'evelyn', 'jocelyn', 'arianna', 'amber', 'isabel', 'melanie', 'diana', 'danielle', 'sierra', 'leslie', 'aaliyah', 'erin', 'amelia', 'molly', 'melissa', 'cla

In [8]:
block_size =3
def build_dataset(words):
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(names)
n1 = int(0.8*len(names))
n2 = int(0.9*len(names))

X, Y = build_dataset(names[:n1])
Xdev, Ydev = build_dataset(names[n1:n2])
Xte, Yte = build_dataset(names[n2:])

torch.Size([184849, 3]) torch.Size([184849])
torch.Size([23094, 3]) torch.Size([23094])
torch.Size([23180, 3]) torch.Size([23180])


In [9]:
g = torch.Generator().manual_seed(42357892)

In [10]:
C = torch.randn((27,10),generator=g)

w1=torch.randn(30,200,generator=g)
b1=torch.randn(200,generator=g)

w2 = torch.randn(200,27,generator=g)
b2 = torch.randn(27,generator=g)

parameters = [C,w1,b1,w2,b2]

for p in parameters:
    p.requires_grad=True
sum(p.nelement() for p in parameters)

11897

In [14]:
for i in range(200000):
    ix = torch.randint(0,X.shape[0],(64,))
    
    # Forward Propogation 
    emb = C[X[ix]] # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 30) @ w1 + b1) # (32, 100)
    logits = h @ w2 + b2 # (32, 27)
    loss = F.cross_entropy(logits, Y[ix])


    # BackWard Propogation

    for p in parameters:
        p.grad = None
    loss.backward()
    lr=0.1
    for p in parameters:
        p.data += -lr*p.grad

print(loss.item())

2.0248565673828125


In [16]:
g = torch.Generator().manual_seed(42357892)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ w1 + b1)
      logits = h @ w2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

berccolliana.
adie.
hae.
keraya.
adioka.
hamattley.
nashakindi.
jai.
dam.
gerat.
umaadvia.
marie.
heavana.
dara.
arilya.
anny.
para.
kris.
olayaylenderija.
canah.
